In [ ]:
import requests

# Replace with the GitHub username
username = "github_username"

# GitHub API base URL
base_url = "https://api.github.com"

# Get list of repositories
repos_url = f"{base_url}/users/{username}/repos"
repos_response = requests.get(repos_url)
repos = repos_response.json()

# Loop through each repository and get commit history
for repo in repos:
    repo_name = repo['name']
    commits_url = f"{base_url}/repos/{username}/{repo_name}/commits"

    # Get commit history for the repository
    commits_response = requests.get(commits_url)
    commits = commits_response.json()

    print(f"Repository: {repo_name}")
    for commit in commits:
        commit_message = commit['commit']['message']
        commit_author = commit['commit']['author']['name']
        commit_date = commit['commit']['author']['date']
        print(f"- {commit_author} ({commit_date}): {commit_message}")

    print("\n")



In [ ]:

import requests
from datetime import datetime, timedelta
from collections import defaultdict


In [ ]:
# Replace with the GitHub username
username = "aryanploxxx"

# Your personal access token from GitHub
token = "enter_your_token_here"

# GitHub API base URL
# base_url = "https://api.github.com"

# Headers with the authentication token
headers = {
    "Authorization": f"token {token}"
}

# Get list of repositories with the authentication token
repos_url = f"{base_url}/users/{username}/repos"
repos_response = requests.get(repos_url, headers=headers)
repos = repos_response.json()

# Ensure that the response is a list
if isinstance(repos, list):
    # Create a dictionary to store commit counts by date
    commit_counts = defaultdict(int)

    # Get today's date and the date 365 days ago
    today = datetime.now().date()
    one_year_ago = today - timedelta(days=365)

    # Loop through each repository and get commit history
    for repo in repos:
        repo_name = repo['name']
        commits_url = f"{base_url}/repos/{username}/{repo_name}/commits"

        # Get commit history for the repository with the authentication token
        commits_response = requests.get(commits_url, headers=headers)
        commits = commits_response.json()

        # Ensure that the commits response is a list
        if isinstance(commits, list):
            # Loop through commits and count them by date
            for commit in commits:
                commit_date = commit['commit']['author']['date']
                commit_date = datetime.strptime(commit_date, "%Y-%m-%dT%H:%M:%SZ").date()

                if one_year_ago <= commit_date <= today:
                    commit_counts[commit_date] += 1

    # Create a list to store the day-wise commit counts
    commit_calendar = []

    for i in range(365):
        day = today - timedelta(days=i)
        commit_calendar.append((day, commit_counts[day]))

    commit_calendar.reverse()

    # Print the day-wise commit data
    for day, count in commit_calendar:
        print(f"Date: {day}, Commits: {count}")
else:
    print("Failed to fetch repositories or user has no public repositories.")


In [ ]:
#visualising the data
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
print(type(commit_calendar))
print(commit_calendar)

In [ ]:

# Create a DataFrame from the commit data
dates, counts = zip(*commit_calendar)
df = pd.DataFrame({
    'date': dates,
    'commits': counts
})

# Convert 'date' column to datetime64 for proper handling
df['date'] = pd.to_datetime(df['date'])

# Extract day of the week and week number
df['day'] = df['date'].dt.strftime('%a')
df['week'] = df['date'].dt.strftime('%U')

# Create a pivot table to get the day-wise commit counts for each week
pivot_df = df.pivot(index='week', columns='day', values='commits').fillna(0)

# Set the days order to match the GitHub-style heatmap
days_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Reorder columns according to days_order
pivot_df = pivot_df[days_order]

# Plot the heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(pivot_df, cmap="Greens", linewidths=0.1, linecolor='white', cbar=False)
plt.title("Yearly Commit Activity")
plt.show()

In [ ]:

# Assuming commit_calendar is a list of (date, commits) tuples
dates, counts = zip(*commit_calendar)
df = pd.DataFrame({
    'date': dates,
    'commits': counts
})

# Convert 'date' column to datetime64 for proper handling
df['date'] = pd.to_datetime(df['date'])

# Extract month, day of the week, and week number
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.weekday  # Monday=0, Sunday=6
df['week'] = df['date'].dt.isocalendar().week

# Create a DataFrame for the heatmap
# 52 weeks and 7 days
heatmap_data = np.zeros((7, 52))

# Populate the heatmap_data
for _, row in df.iterrows():
    heatmap_data[row['day'], row['week'] - 1] += row['commits']

# Convert to DataFrame for easier plotting
heatmap_df = pd.DataFrame(heatmap_data, columns=range(1, 53), index=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# Plot the heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(heatmap_df, cmap="Greens", linewidths=0.1, linecolor='white', cbar=True)
plt.title("Yearly Commit Activity")
plt.xlabel("Week of the Year")
plt.ylabel("Day of the Week")
plt.show()